In [144]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, scale
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.special as sc

In [21]:
df = pd.read_csv('SAheart.csv')
df.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


In [22]:
label_encoder = LabelEncoder()
df['famhist'] = label_encoder.fit_transform(df['famhist'])
df.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,1,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,0,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,1,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,1,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,1,60,25.99,57.34,49,1


In [137]:
null_model = lm.LogisticRegression(C=1e6,solver='newton-cg',intercept_scaling=1e3)
y = df['chd']
dummy_var = np.zeros_like(y).reshape(-1, 1)
null_model.fit(dummy_var, y)
null_model.coef_, null_model.intercept_

(array([[0.]]), array([-0.6352532]))

In [60]:
features = df.drop(columns=['chd']).columns
reduced_models = []
for i, f in enumerate(features):
    lr_model = lm.LogisticRegression()
    lr_model.fit(np.array(df[f]).reshape(-1, 1), y)
    reduced_models.append(lr_model)

In [99]:
def log_likelihood(probs, target):
    return np.sum(target * np.log(probs) + (1 - target) * np.log(1 - probs))
def likelihood_ratio_test(logl_reduced, logl_full, df):
    likelihood_ratio = -2 * (logl_reduced - logl_full)
    return sc.gammaincc(df / 2, likelihood_ratio / 2)

In [96]:
null_model_ll = log_likelihood(null_model.predict_proba(dummy_var)[:, 1], y)
print(np.mean(model.predict(dummy_var) != y), f)
np.mean(y==0)

0.3463203463203463 age


0.6536796536796536

In [138]:
reduced_models_ll = []
lratio_test_pvalues = []
for i, f in enumerate(features):
    model = lm.LogisticRegression(C=1e6,solver='newton-cg',intercept_scaling=1e3)
    model.fit(np.array(df[f]).reshape(-1, 1), y)
    model_ll = log_likelihood(model.predict_proba(np.array(df[features[i]]).reshape(-1, 1))[:, 1], y)
    print(np.mean(model.predict(np.array(df[features[i]]).reshape(-1, 1)) == y), f)
    reduced_models_ll.append(model_ll)
    p_value = likelihood_ratio_test(null_model_ll, model_ll, 1)
    lratio_test_pvalues.append(p_value)

0.6666666666666666 sbp
0.7012987012987013 tobacco
0.6666666666666666 ldl
0.6774891774891775 adiposity
0.6536796536796536 famhist
0.6536796536796536 typea
0.6536796536796536 obesity
0.6515151515151515 alcohol
0.6796536796536796 age


In [139]:
lratio_test_pvalues

[4.183864256534665e-05,
 1.202478016291409e-10,
 1.6830830956138175e-08,
 2.505507012941752e-08,
 4.9370933438074925e-09,
 0.025586907377011928,
 0.03234609691484565,
 0.18425399732316508,
 4.4963940717236015e-17]

In [140]:
features_sorted = features[np.argsort(lratio_test_pvalues)]

In [141]:
model = lm.LogisticRegression(C=1e6,solver='newton-cg',intercept_scaling=1e3)
model.fit(np.array(df[features_sorted[0]]).reshape(-1, 1), y)
ll_prev = log_likelihood(model.predict_proba(np.array(df[features_sorted[0]]).reshape(-1, 1))[:, 1], y)
p_value = likelihood_ratio_test(null_model_ll, model_ll, 1)
for i in range(2, len(features)):
    model = lm.LogisticRegression(C=1e6,solver='newton-cg',intercept_scaling=1e3)
    print(i, features_sorted[:i])
    model.fit(df[features_sorted[:i]], y)
    model_ll = log_likelihood(model.predict_proba(df[features_sorted[:i]])[:, 1], y)
    print(np.mean(model.predict(df[features[:i]]) == y), f)
    p_value = likelihood_ratio_test(ll_prev, model_ll, 1)
    print(p_value)
    if p_value > 0.05:
        break
    else:
        ll_prev = model_ll

2 Index(['age', 'tobacco'], dtype='object')
0.3463203463203463 age
0.0014221942320854625
3 Index(['age', 'tobacco', 'famhist'], dtype='object')
0.3463203463203463 age
7.743624951254233e-06
4 Index(['age', 'tobacco', 'famhist', 'ldl'], dtype='object')
0.3463203463203463 age
0.001615904923532378
5 Index(['age', 'tobacco', 'famhist', 'ldl', 'adiposity'], dtype='object')
0.3463203463203463 age
0.6245834490856955


In [142]:
model.coef_

array([[ 0.04645584,  0.0808489 ,  0.92586288,  0.17759533, -0.0092982 ]])

In [166]:
df_scaled = scale(df[features])
logreg_l1 = LogisticRegression(C=0.05,intercept_scaling=1e3,penalty='l1', solver='liblinear')
logreg_l1.fit(df_scaled, y)   

LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1000.0, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [167]:
logreg_l1.coef_

array([[0.        , 0.20990575, 0.17620137, 0.        , 0.25698461,
        0.070586  , 0.        , 0.        , 0.47871709]])

In [169]:
features[logreg_l1.coef_.flatten() != 0]

Index(['tobacco', 'ldl', 'famhist', 'typea', 'age'], dtype='object')